In [1]:
import re
import json
import os.path
import pandas as pd

from IPython.display import display
from tqdm import tqdm
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=0)

from gensim.models import Phrases
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import spacy
from spacy.tokens import Doc
print(spacy.__version__)
nlp = spacy.load("en_core_web_sm")

3.0.6


/opt/conda/lib/python3.8/site-packages/packaging/version.py:127: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  warnings.warn(


In [2]:
all_levels = pd.read_csv('all_levels.csv', low_memory=False)
print(len(all_levels))
display(all_levels.head())
# display(all_levels['topic'].value_counts()) # 128 prompts
texts = all_levels['text'].tolist()
print(len(texts))

# This elimates all student responses that are not automatically interpreted as strings by Pandas.This removes about 1,500 responses.
# A cursory investigation suggests that all these 'non-string' responses are NaN values, so useless.
texts = [text for text in texts if type(text) == str]

texts = [re.sub(r"\s+", r" ", text) for text in texts]

for i, text in enumerate(texts[0:20000]):
    if type(text) != str:
        print(text, i)
# print(texts[0:5])

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


549281


,Unnamed: 0,id,lvl,unit,author_id,author_nationality,topic,topic_id,date,grade,text
0,0,C18217,1,7,20967052,cn,Writing labels for a clothing store,22440,2012-04-20 21:12:14.890,79,Date:monday 11th. Time:9.30 am. From:Margar...
1,1,C18541,1,1,21016113,cn,Introducing yourself by email,3535,2011-12-24 03:50:49.100,85,"Dear teacher, My name's Yi Zhao,English is poo..."
2,2,C18648,1,1,20967075,cn,Introducing yourself by email,3535,2012-04-20 08:53:55.087,90,"My name's Henry Hong, I was born in Nanjing o..."
3,3,C20184,1,1,18898793,cn,Introducing yourself by email,3535,2011-12-14 08:54:48.380,95,"Dear Sir, I'm Jianwen Zhang, from a little tow..."
4,4,C20185,1,2,18898793,cn,Taking inventory in the office,9820,2011-12-26 09:49:48.140,98,"Dear Ms Thomas, There are thirteen computers a..."


549281


In [3]:
# Run corpus through the spaCy pipeline

def process_docs(texts):
    docs = list(tqdm(nlp.pipe(texts))) # 20 minutes or so...
    return docs

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# Remove punctuation, stop words, and digits
# Make all words lowercase
# We can add to the stop words list here, or use other measures to eliminate uninteresting words (TF-IDF, Dispersion)

def reduce_tokens(docs):
    processed_docs = []
    for doc in docs:
        processed_doc = []
        for token in doc:
            if not token.is_punct and not token.is_stop and not token.is_digit:
                processed_doc.append(token.lemma_.lower())
        processed_docs.append(processed_doc)
    return processed_docs

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# Compute bigrams.
# Add bigrams to docs (only ones that appear 20 times or more).

def compute_bigrams(processed_docs):
    bigram = Phrases(processed_docs, min_count=20)
    for idx in range(len(processed_docs)):
        for token in bigram[processed_docs[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                processed_docs[idx].append(token)
    return processed_docs

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
import os.path # Directories, gotta have em.

processed_docs_filename = "processed_docs.json"

if os.path.isfile(processed_docs_filename):
    print('Retrieving processed corpus from file')
    with open('processed_docs.json') as f:
        processed_docs = json.load(f)
    print('Retrieved')
else:
    print('Nothing found.')
    spacy_docs = process_docs(texts)
    reduced_docs = reduce_tokens(spacy_docs)
    processed_docs = compute_bigrams(reduced_docs)
    print('Corpus Processed.')
    

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Retrieving processed corpus from file
Retrieved


In [7]:
# Remove rare and common tokens.
# Create a dictionary representation of the documents.
dictionary = Dictionary(processed_docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Number of unique tokens: 24919
Number of documents: 547637


In [ ]:
# Train LDA model.
# Set training parameters.
num_topics = 20
chunksize = 600000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# print(model.print_topics())
# doc_lda = model[corpus]

p = gensimvis.prepare(model, corpus, dictionary)
p